In [1]:
"""
If you want to use new map results, then go to tournament_reduce.py, and replace the extract() method with the following code. 
This will save the map results for use in this notebook. Once those are saved, switch the original code back. 
"""
"""

async def extract(
    self, results: list[LookupOutput], district: District, term: str
):
    # We first map extraction across all pages.
    results = []
    empty_results = []
    map_pickle = []
    async for r in super().extract(pages, district, term):
        map_pickle.append(r)
        if (r.output is not None) and r.output.extracted_text:
            results.append(r)
        else:
            empty_results.append(r)
            
    # Load existing data if the file already exists
    try:
        with open("./ryurtyn/map_results_11_14_0.dat", "rb") as f:
            existing_data = pickle.load(f)
    except FileNotFoundError:
        existing_data = {}  # If the file doesn't exist, initialize an empty dictionary

    try:
        with open("./ryurtyn/districts_11_14_0.dat", "rb") as f:
            existing_data_district = pickle.load(f)
    except FileNotFoundError:
        existing_data_district = {} 
    # Assuming district.short_name and results are defined somewhere in your code
    new_data = {district.short_name: map_pickle}
    new_data_district = {district.short_name: district}

    # Merge the existing data with the new data
    existing_data.update(new_data)
    existing_data_district.update(new_data_district)

    # Write the updated data back to the file
    with open("./ryurtyn/map_results_11_14_0.dat", "wb") as f:
        pickle.dump(existing_data, f)

    with open("./ryurtyn/districts_11_14_0.dat", "wb") as f:
        pickle.dump(existing_data_district, f)

    # Ensure that we yield one empty result to handle case when the expected output is None
    if len(empty_results) != 0:
        yield empty_results[0]

"""

'\n\nasync def extract(\n    self, results: list[LookupOutput], district: District, term: str\n):\n    # We first map extraction across all pages.\n    results = []\n    empty_results = []\n    map_pickle = []\n    async for r in super().extract(pages, district, term):\n        map_pickle.append(r)\n        if (r.output is not None) and r.output.extracted_text:\n            results.append(r)\n        else:\n            empty_results.append(r)\n            \n    # Load existing data if the file already exists\n    try:\n        with open("./ryurtyn/map_results_11_14_0.dat", "rb") as f:\n            existing_data = pickle.load(f)\n    except FileNotFoundError:\n        existing_data = {}  # If the file doesn\'t exist, initialize an empty dictionary\n\n    try:\n        with open("./ryurtyn/districts_11_14_0.dat", "rb") as f:\n            existing_data_district = pickle.load(f)\n    except FileNotFoundError:\n        existing_data_district = {} \n    # Assuming district.short_name and res

In [2]:
import pickle
from zoning.term_extraction.extract.tournament_tester import TestTournamentReduceExtractor, tournament_reduce_test
from zoning.term_extraction.eval_results import clean_string_units
import asyncio
import pandas as pd

sk


/Users/rusy/Documents/Cornell/fa23/Zoning/zoning-gpt/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# load the map outputs
with open("map_results_11_14_0.dat", "rb") as f:
    map_outputs = pickle.load(f)

# load the dictionary of district objects
with open("districts_11_14_0.dat", "rb") as f:
    districts = pickle.load(f)

In [4]:
gt = pd.read_csv("../data/ground_truth.csv")
gt_first_30 = gt.head(30)

In [5]:
gt_value_exists = gt_first_30[gt_first_30["min_lot_size_gt"].notnull()]
gt_value_not_exist = gt_first_30[gt_first_30["min_lot_size_gt"].isnull()]

In [6]:
# Define an asynchronous function that executes the coroutine object
async def execute_tournament_test(inputs, term, district, town):
    # Your coroutine object (replace this with your actual coroutine)
    async def my_coroutine():
        return await tournament_reduce_test(inputs, term, district, 1, town)
    # Execute the coroutine object using await
    return await my_coroutine()

In [7]:
"""
When you're ready to run the experiment, replace the extract method of tournament with this code.
This ensure that you don't map extract, and just run tournamnet on your pair of elements
"""

"""
async def extract(
    self, pages: list[LookupOutput], district: District, term: str
):
    results = pages
    for result in await tournament_reduce_test(results, term, district, self.k):
        yield result
"""

'\nasync def extract(\n    self, pages: list[LookupOutput], district: District, term: str\n):\n    results = pages\n    for result in await tournament_reduce_test(results, term, district, self.k):\n        yield result\n'

Comparing Results Where GT doesn't exist

In [10]:
total_length = 0
total_selected_index = 0
total_selected_none = 0

for d in gt_value_not_exist["district"]:
    district_short_name = gt_first_30[gt_first_30["district"] == d]["district_abb"].values[0]
    district = districts[district_short_name]
    inputs = map_outputs[district_short_name]
    town = gt_first_30[gt_first_30["district"] == d]["town"].values[0]
    term = "min_lot_size"
    print("District: ", district)

    selected_index = 0
    selected_none = 0
    total_length += len(inputs)
    for a in range(len(inputs)):
        i = inputs[a]
        if i.output and i.output.extracted_text:
            for b in range(a + 1, len(inputs)):
                j = inputs[b]
                if j.output and j.output.extracted_text:
                    res = await execute_tournament_test([i, j], term, district, town)
                    if res == -1:
                        selected_none += 1
                        total_selected_none += 1
                        
                    elif res == 1 or res == 0:
                        selected_index += 1
                        total_selected_index += 1
                    

    print("Selected None: ", selected_none)
    print("Selected Index: ", selected_index)
    print("Accuracy: ", selected_none / len(inputs))

print("Final Selected None: ", total_selected_none)
print("Final Accuracy: ", total_selected_none / total_length)

District:  full_name='Andover Lake' short_name='AL'


/Users/rusy/Documents/Cornell/fa23/Zoning/zoning-gpt/zoning/term_extraction/extract/utils.py:179: UserWarning: Phrase AL- One Hundred and twenty thousand (120,000) square Feet or 2.75 acres, calculated including the Access way was not in the supplied. document: 
NEW PAGE 22
4.9.1
Authorization - The Planning and Zoning Commission may grant Special Permits in
accordance with the provisions of this section, in lieu of the requirements of Section
23, to allow the construction of a single-family dwelling with permitted accessory
buildings or uses on Rear Lots in the ARD and AL Zones. Rear Lots included in
proposed subdivisions shall be subject to the provisions of this section. All Rear Lots
require a Special Permit from the Planning and Zoning Commission.
4.9.2
Definition "Rear Lot" - A lot in one of the aforesaid Zones with less than the required
Lot Width on an accepted Town road or a new road depicted on an approved
subdivision.
4.9.3
Requirements - Each Rear Lot shall comply with all 

Selected None:  0
Selected Index:  28
Accuracy:  0.0
District:  full_name='Technology Development' short_name='TD'
Selected None:  0
Selected Index:  3
Accuracy:  0.0
District:  full_name='Industrial Park' short_name='IP'


/Users/rusy/Documents/Cornell/fa23/Zoning/zoning-gpt/zoning/term_extraction/extract/utils.py:197: UserWarning: Phrase CELL (1, 1): 
Minimum lot area was found more than once in the document.
  warnings.warn(f"Phrase {phrase} was found more than once in the document.")


Selected None:  0
Selected Index:  28
Accuracy:  0.0
District:  full_name='Village Overlay' short_name='VDO'
Selected None:  0
Selected Index:  15
Accuracy:  0.0
District:  full_name='I-1 General Industry' short_name='IND-1'


...

Selected None:  0
Selected Index:  6
Accuracy:  0.0
District:  full_name='R-3 Residential' short_name='R-3'


/Users/rusy/Documents/Cornell/fa23/Zoning/zoning-gpt/zoning/term_extraction/extract/utils.py:197: UserWarning: Phrase CELL (4, 1): 
R-3 was found more than once in the document.
  warnings.warn(f"Phrase {phrase} was found more than once in the document.")


Selected None:  0
Selected Index:  15
Accuracy:  0.0
District:  full_name='Aquifer Protection' short_name='AZ'


/Users/rusy/Documents/Cornell/fa23/Zoning/zoning-gpt/zoning/term_extraction/extract/utils.py:179: UserWarning: Phrase +1. Not more than one single-family dwelling, with or without accessory apartment on a lot of two acres or more. was not in the supplied. document: 
NEW PAGE 17
7.
All work shall be performed in accordance with the Road Foreman's requirements.
+8. Driveway apron design should conform to standards set by the Chaplin Board of Selectmen.
8a. Bituminous concrete berm-type driveway aprons (entrances) shall be installed by the applicant
for each building lot in order to check erosion, prevent surface flooding, minimize maintenance, and
control stormwater runoff. All such driveway entrances shall be designed and located to provide safe and
convenient vehicular access. No driveway shall be located SO as to present possible traffic safety
problems. Said driveway apron shall be a minimum of ten (10) feet wide at the right-of-way line and
flare out thereafter some two and one-half

Selected None:  0
Selected Index:  15
Accuracy:  0.0
Final Selected None:  0
Final Accuracy:  0.0


Comparing Results Where GT exists

In [14]:
total_length = 0
total_selected_correct = 0
total_selected_incorrect = 0

for d in gt_value_exists["district"]:
    district_short_name = gt_first_30[gt_first_30["district"] == d]["district_abb"].values[0]
    district = districts[district_short_name]
    inputs = map_outputs[district_short_name]
    town = gt_first_30[gt_first_30["district"] == d]["town"].values[0]

    print(d)
    true_val = gt_first_30[gt_first_30["district"] == d]["min_lot_size_gt"].values[0]
    try:
        true_val = int(true_val)
    except ValueError:
        print("skipping ", d, "true value is", true_val)
        continue

    term = "min_lot_size"
    print("District: ", district)

    selected_correct = 0
    selected_incorrect = 0
    num_results = 0
    # total_length += len(inputs)
    # find which one is true answer
    true_index = None
    for a in range(len(inputs)):
        i = inputs[a]
        if i.output and i.output.answer:
            # print(i.output.answer)
            i_answer_clean = clean_string_units(i.output.answer)
            print("GT answer: ", true_val, "Cleaned Answer: ", i_answer_clean, "Raw Answer: ", i.output.answer)
            if i_answer_clean:
                i_answer_clean = int(i_answer_clean[0])
                if i_answer_clean == true_val:
                    true_index = a
                    continue
    # if you found right answer, that becomes the 0th index you pass in to tournament
    if true_index is not None:
        for a in range(len(inputs)):
            if a != true_index:
                i = inputs[a]
                if i.output and i.output.extracted_text:
                    res = await execute_tournament_test([inputs[true_index], i], term, district, town)
                    total_length += 1
                    num_results += 1
                    if res == 0:
                        selected_correct += 1
                        total_selected_correct += 1 
                    if res == 1:
                        selected_incorrect += 1
                        total_selected_incorrect += 1
        print("Selected Correct: ", selected_correct)
        print("Selected Incorrect: ", selected_incorrect)
        print("Accuracy: ", selected_correct / num_results)
    else:
        print("true answer not found in answers?? check this ")


print("Final Selected Correct: ", total_selected_correct)
print("Final Selected Incorrect: ", total_selected_incorrect)
print("Final Accuracy: ", total_selected_correct / total_length)

AA Residence
District:  full_name='AA Residence' short_name='AA'
GT answer:  30000 Cleaned Answer:  [30000.0] Raw Answer:  30,000 sq ft
GT answer:  30000 Cleaned Answer:  [20000.0] Raw Answer:  20,000 sq ft
GT answer:  30000 Cleaned Answer:  [30000.0] Raw Answer:  30,000 sq ft
GT answer:  30000 Cleaned Answer:  [653400.0] Raw Answer:  15 acres
GT answer:  30000 Cleaned Answer:  [12500.0] Raw Answer:  12,500 sq ft
GT answer:  30000 Cleaned Answer:  [7500.0] Raw Answer:  7,500 sq ft
GT answer:  30000 Cleaned Answer:  [187308.0] Raw Answer:  4.3 acres
Selected Correct:  6
Selected Incorrect:  0
Accuracy:  1.0
Restricted Business
District:  full_name='Restricted Business' short_name='B-1'
GT answer:  43560 Cleaned Answer:  [87120.0] Raw Answer:  2 acres
GT answer:  43560 Cleaned Answer:  [43560.0] Raw Answer:  1 acre
GT answer:  43560 Cleaned Answer:  [40000.0] Raw Answer:  40,000 sq ft
GT answer:  43560 Cleaned Answer:  [130000.0] Raw Answer:  130,000 sq ft
GT answer:  43560 Cleaned Answe

/Users/rusy/Documents/Cornell/fa23/Zoning/zoning-gpt/zoning/term_extraction/extract/utils.py:179: UserWarning: Phrase The individual lots in Conservation Developments may be reduced from the conventional area, except that lot shall have a minimum lot size of 40,000 sq ft was not in the supplied. document: 
NEW PAGE 68
in keeping with the general character of Barkhamsted. The Commission shall be guided by criteria
outlined in Article VIII in reviewing and permitting a residential cluster special exception. A site plan
shall be prepared for a residential cluster application in accordance with the requirements of Article VII.
Applicants are encouraged to present preliminary plans to the Commission for informal review prior to
submitting a full application.
A. Permitted uses. Any building or: buildings designed and constructed or rehabilitated for use as
residential dwelling units shall be permitted. Such residential cluster shall be allowed in any residential
zoning district.
B. Minimum l

Selected Correct:  6
Selected Incorrect:  0
Accuracy:  1.0
Town Center Perimeter Overlay
District:  full_name='Town Center Perimeter Overlay' short_name='TCD-P'
GT answer:  10000 Cleaned Answer:  [10000.0] Raw Answer:  10,000 sq ft
GT answer:  10000 Cleaned Answer:  [217800.0] Raw Answer:  5 acres
GT answer:  10000 Cleaned Answer:  [100000.0] Raw Answer:  100,000 sq ft
GT answer:  10000 Cleaned Answer:  [43560.0] Raw Answer:  1 acre
GT answer:  10000 Cleaned Answer:  [120000.0] Raw Answer:  120,000 sq ft
Selected Correct:  4
Selected Incorrect:  0
Accuracy:  1.0
Industrial
District:  full_name='Industrial' short_name='IND'
GT answer:  87120 Cleaned Answer:  [87120.0] Raw Answer:  2 acres
GT answer:  87120 Cleaned Answer:  [87120.0] Raw Answer:  2 acres
GT answer:  87120 Cleaned Answer:  [20000.0] Raw Answer:  20,000 sq ft
GT answer:  87120 Cleaned Answer:  [10000.0] Raw Answer:  10,000 sq ft
GT answer:  87120 Cleaned Answer:  [30000.0] Raw Answer:  30,000 sq ft
GT answer:  87120 Cleane

/Users/rusy/Documents/Cornell/fa23/Zoning/zoning-gpt/zoning/term_extraction/extract/utils.py:197: UserWarning: Phrase CELL (1, 1): 
Minimum Lot Size was found more than once in the document.
  warnings.warn(f"Phrase {phrase} was found more than once in the document.")
/Users/rusy/Documents/Cornell/fa23/Zoning/zoning-gpt/zoning/term_extraction/extract/utils.py:179: UserWarning: Phrase the minimum area of any lot on which such animals are kept shall be 1.5 acres for one animal, plus 1.0 additional acre for each additional animal. was not in the supplied. document: 
NEW PAGE 61
6. A shed which is less than ten (10) feet in height at the roof ridge and less than eighty (80)
square feet in ground floor area may be located within a required side or rear yard, but no
closer than ten (10) feet to any lot line.
6.4.
Keeping of Equine Animals
1. Equine animals (including horses, donkeys, burros and their young) may be kept in any
zone, provided the minimum area of any lot on which such animals a